In [1]:
#Importing libraries
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from os import getcwd
from tensorflow import keras
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Embedding, Dropout
from sklearn import preprocessing
import datetime as dt

In [2]:
#Reading data
url = "https://hub.mph.in.gov/dataset/bd08cdd3-9ab1-4d70-b933-41f9ef7b809d/resource/afaa225d-ac4e-4e80-9190-f6800c366b58/download/covid_report_county_date.xlsx?raw=true"
cases_data = pd.read_excel(url)

# population_data = pd.read_csv(f"{getcwd()}/drive/My Drive/Colab Notebooks/COVID forecasting/indiana countywise population.csv")

population_data = pd.read_csv(f"{getcwd()}/drive/My Drive/Colab Notebooks/COVID forecasting/population density of indiana county.csv")

#Data preprocessing
population_data["County"] = population_data["County"].str.upper()
population_data["County"] = population_data["County"].str.replace('.', '')
population_data["County"] = population_data["County"].str.replace(' ', '')
cases_data["COUNTY_NAME"] = cases_data["COUNTY_NAME"].str.replace(' ','')

data = pd.merge(cases_data, population_data, how = 'inner', left_on = 'COUNTY_NAME', right_on = 'County')
data = data[["COUNTY_NAME", "DATE", "COVID_COUNT",  "COVID_DEATHS", "COVID_TEST", "Population", "Area", "Population Density"]]
data

,COUNTY_NAME,DATE,COVID_COUNT,COVID_DEATHS,COVID_TEST,Population,Area,Population Density
0,ADAMS,2020-02-29,0,0,0,34387.0,339,101.436578
1,ADAMS,2020-03-02,0,0,0,34387.0,339,101.436578
2,ADAMS,2020-03-03,0,0,0,34387.0,339,101.436578
3,ADAMS,2020-03-04,0,0,0,34387.0,339,101.436578
4,ADAMS,2020-03-05,0,0,1,34387.0,339,101.436578
...,...,...,...,...,...,...,...,...
11679,WHITLEY,2020-07-01,1,0,42,33292.0,336,99.083333
11680,WHITLEY,2020-07-02,7,1,25,33292.0,336,99.083333
11681,WHITLEY,2020-07-03,2,0,8,33292.0,336,99.083333
11682,WHITLEY,2020-07-04,2,0,3,33292.0,336,99.083333


In [3]:
data.describe()

,COVID_COUNT,COVID_DEATHS,COVID_TEST,Population,Area,Population Density
count,11684.000000,11684.000000,11684.000000,11684.000000,11684.000000,11684.000000
mean,4.136511,0.214396,45.069497,70436.967391,389.434783,177.640432
std,15.057564,0.914656,116.442069,116127.448828,95.500309,280.278721
min,0.000000,0.000000,0.000000,6128.000000,86.000000,21.807882
25%,0.000000,0.000000,3.000000,21494.500000,360.250000,57.580223
50%,1.000000,0.000000,11.000000,33844.000000,396.500000,82.635162
75%,2.000000,0.000000,36.000000,69188.250000,444.250000,169.554527
max,432.000000,16.000000,1733.000000,903393.000000,657.000000,2281.295455


In [4]:
county_name = data["COUNTY_NAME"].unique()
county_name

array(['ADAMS', 'ALLEN', 'BARTHOLOMEW', 'BENTON', 'BLACKFORD', 'BOONE',
       'BROWN', 'CARROLL', 'CASS', 'CLARK', 'CLAY', 'CLINTON', 'CRAWFORD',
       'DAVIESS', 'DEARBORN', 'DECATUR', 'DEKALB', 'DELAWARE', 'DUBOIS',
       'ELKHART', 'FAYETTE', 'FLOYD', 'FOUNTAIN', 'FRANKLIN', 'FULTON',
       'GIBSON', 'GRANT', 'GREENE', 'HAMILTON', 'HANCOCK', 'HARRISON',
       'HENDRICKS', 'HENRY', 'HOWARD', 'HUNTINGTON', 'JACKSON', 'JASPER',
       'JAY', 'JEFFERSON', 'JENNINGS', 'JOHNSON', 'KNOX', 'KOSCIUSKO',
       'LAGRANGE', 'LAKE', 'LAPORTE', 'LAWRENCE', 'MADISON', 'MARION',
       'MARSHALL', 'MARTIN', 'MIAMI', 'MONROE', 'MONTGOMERY', 'MORGAN',
       'NEWTON', 'NOBLE', 'OHIO', 'ORANGE', 'OWEN', 'PARKE', 'PERRY',
       'PIKE', 'PORTER', 'POSEY', 'PULASKI', 'PUTNAM', 'RANDOLPH',
       'RIPLEY', 'RUSH', 'SCOTT', 'SHELBY', 'SPENCER', 'STJOSEPH',
       'STARKE', 'STEUBEN', 'SULLIVAN', 'SWITZERLAND', 'TIPPECANOE',
       'TIPTON', 'UNION', 'VANDERBURGH', 'VERMILLION', 'VIGO', 'WABASH',
   

In [5]:
print('Length of the sample: ', len(data))

Length of the sample:  11684


In [6]:
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

In [7]:
#Scaling the data
scaler = preprocessing.StandardScaler()
scaled_data = scaler.fit_transform(data[['COVID_COUNT', 'COVID_DEATHS', 'COVID_TEST', 'Population', 'Area', 'Population Density']])

data["COVID_COUNT"] = scaled_data[:, 0]
data["COVID_DEATHS"] = scaled_data[:, 1]
data["COVID_TEST"] = scaled_data[:, 2]
data["Population"] = scaled_data[:, 3]
data["Area"] = scaled_data[:, 4]
data["Population Density"] = scaled_data[:, 5]

data

,COUNTY_NAME,DATE,COVID_COUNT,COVID_DEATHS,COVID_TEST,Population,Area,Population Density
0,ADAMS,2020-02-29,-0.274725,-0.234411,-0.387072,-0.310448,-0.528134,-0.271898
1,ADAMS,2020-03-02,-0.274725,-0.234411,-0.387072,-0.310448,-0.528134,-0.271898
2,ADAMS,2020-03-03,-0.274725,-0.234411,-0.387072,-0.310448,-0.528134,-0.271898
3,ADAMS,2020-03-04,-0.274725,-0.234411,-0.387072,-0.310448,-0.528134,-0.271898
4,ADAMS,2020-03-05,-0.274725,-0.234411,-0.378483,-0.310448,-0.528134,-0.271898
...,...,...,...,...,...,...,...,...
11679,WHITLEY,2020-07-01,-0.208310,-0.234411,-0.026362,-0.319877,-0.559549,-0.280294
11680,WHITLEY,2020-07-02,0.190178,0.858944,-0.172363,-0.319877,-0.559549,-0.280294
11681,WHITLEY,2020-07-03,-0.141896,-0.234411,-0.318365,-0.319877,-0.559549,-0.280294
11682,WHITLEY,2020-07-04,-0.141896,-0.234411,-0.361307,-0.319877,-0.559549,-0.280294


In [8]:
# reshape to [samples, time_steps, n_features]
time_steps = 21
BATCH_SIZE = 1024
X_train, y_train = [], []

for county in county_name:
  filtered_data = data.loc[data["COUNTY_NAME"] == county]
  train_size = len(filtered_data)
  filtered_train = filtered_data.iloc[0:train_size]
  
  filtered_X_train, filtered_y_train = create_dataset(filtered_train[['COVID_COUNT', 'COVID_DEATHS', 'COVID_TEST', 'Population', 'Area', 'Population Density']], 
                                                      filtered_train[['COVID_COUNT', 'COVID_DEATHS']], time_steps)

  if(len(X_train) == 0):
    X_train, y_train = filtered_X_train, filtered_y_train
  else:
    X_train = np.vstack((X_train, filtered_X_train))
    y_train = np.vstack((y_train, filtered_y_train))

print(X_train.shape, y_train.shape)

(9752, 21, 6) (9752, 2)


In [9]:
# Loading model
model = keras.models.load_model(f"{getcwd()}/drive/My Drive/Colab Notebooks/COVID forecasting/checkpoint.h5")
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_24 (LSTM)               (None, 21, 1024)          4222976   
_________________________________________________________________
dropout_24 (Dropout)         (None, 21, 1024)          0         
_________________________________________________________________
lstm_25 (LSTM)               (None, 21, 512)           3147776   
_________________________________________________________________
dropout_25 (Dropout)         (None, 21, 512)           0         
_________________________________________________________________
lstm_26 (LSTM)               (None, 21, 256)           787456    
_________________________________________________________________
dropout_26 (Dropout)         (None, 21, 256)           0         
_________________________________________________________________
lstm_27 (LSTM)               (None, 64)               

In [10]:
# #Defining Model
# model = Sequential()
# model.add(LSTM(BATCH_SIZE, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
# model.add(Dropout(0.1))
# model.add(LSTM(512, return_sequences=True))
# model.add(Dropout(0.1))
# model.add(LSTM(256, return_sequences=True))
# model.add(Dropout(0.1))
# model.add(LSTM(64, return_sequences=False))
# model.add(Dropout(0.1))
# model.add(Dense(32))
# model.add(Dense(2))

# model.summary()

In [11]:
model.compile(loss = 'mean_squared_error', optimizer = 'rmsprop', metrics = ['mse', 'accuracy'])

#1500 epochs for training model

history = model.fit(
    X_train, y_train,
    epochs = 50,
    batch_size = BATCH_SIZE
)

Epoch 1/50
10/10 [==============================] - 2s 154ms/step - loss: 0.0094 - mse: 0.0094 - accuracy: 0.9459
Epoch 2/50
10/10 [==============================] - 2s 151ms/step - loss: 0.0084 - mse: 0.0084 - accuracy: 0.9571
Epoch 3/50
10/10 [==============================] - 2s 152ms/step - loss: 0.0077 - mse: 0.0077 - accuracy: 0.9592
Epoch 4/50
10/10 [==============================] - 2s 151ms/step - loss: 0.0064 - mse: 0.0064 - accuracy: 0.9599
Epoch 5/50
10/10 [==============================] - 2s 151ms/step - loss: 0.0072 - mse: 0.0072 - accuracy: 0.9606
Epoch 6/50
10/10 [==============================] - 2s 150ms/step - loss: 0.0070 - mse: 0.0070 - accuracy: 0.9649
Epoch 7/50
10/10 [==============================] - 2s 152ms/step - loss: 0.0075 - mse: 0.0075 - accuracy: 0.9556
Epoch 8/50
10/10 [==============================] - 2s 151ms/step - loss: 0.0057 - mse: 0.0057 - accuracy: 0.9578
Epoch 9/50
10/10 [==============================] - 2s 151ms/step - loss: 0.0062 - mse: 

In [ ]:
# fig, (ax1, ax2) = plt.subplots(1, 2)

# fig.suptitle('Evaluation Metrics')
# fig.set_size_inches(15, 6)

# ax1.plot(history.history['accuracy'])
# ax1.title.set_text('Model Accuracy')
# ax1.set_xlabel('Epochs')
# ax1.set_ylabel('Units')

# ax2.plot(history.history['loss'])
# ax2.title.set_text('Model Loss')
# ax2.set_xlabel('Epochs')
# ax2.set_ylabel('Units')

# plt.show()

In [12]:
checkpoint_path = f"{getcwd()}/drive/My Drive/Colab Notebooks/COVID forecasting/checkpoint.h5"
model.save(checkpoint_path)

In [13]:
for county in county_name:
  demo_data = data[data['COUNTY_NAME'] == county]
  demo_data = demo_data[['COVID_COUNT', 'COVID_DEATHS', 'COVID_TEST', 'Population', 'Area', 'Population Density']]
  demo_data = demo_data.tail(time_steps + 1)

  demo_X_test, demo_y_test = create_dataset(demo_data[['COVID_COUNT', 'COVID_DEATHS', 'COVID_TEST', 'Population', 'Area', 'Population Density']], 
                                                      demo_data[['COVID_COUNT', 'COVID_DEATHS']], time_steps)

  predictions = np.concatenate((model.predict(demo_X_test), np.zeros((1, 4))), axis = 1)
  predictions = np.intc(scaler.inverse_transform(predictions))
  # print(county, ':', predictions[:, 0:2])

  transformed_demo_data = np.intc(scaler.inverse_transform(demo_data))
  demo_data['COVID_COUNT'] = transformed_demo_data[:, 0]
  demo_data['COVID_DEATHS'] = transformed_demo_data[:, 1]
  demo_data = demo_data[['COVID_COUNT', 'COVID_DEATHS']]
  demo_data = np.concatenate((demo_data, predictions[:, 0:2]), axis = 0)

  date_data = data[['DATE']]
  date_data = date_data.tail(time_steps + 2)
  date_data['DATE'] = pd.to_datetime(date_data['DATE']).apply(pd.DateOffset(1))

  fig, (ax1, ax2) = plt.subplots(1, 2)
  fig.suptitle('COVID FORECASTING for ' + county + ' County')
  fig.set_size_inches(15, 6)
  plt.setp(ax1.xaxis.get_majorticklabels(), rotation=90)
  plt.setp(ax2.xaxis.get_majorticklabels(), rotation=90)

  ax1.plot(date_data, demo_data[:, 0], linestyle='-', marker='o', color = 'red', label = 'COVID CASES')
  ax1.grid(True)
  ax1.set_xlabel('Time')
  ax1.set_ylabel('Count')
  ax1.legend()

  ax2.plot(date_data, demo_data[:, 1], linestyle='-', marker='o', color = 'blue', label = 'COVID DEATHS')
  ax2.grid(True)
  ax2.set_xlabel('Time')
  ax2.set_ylabel('Count')
  ax2.legend()
  # plt.show()

  fig.savefig(f"{getcwd()}/drive/My Drive/Colab Notebooks/COVID forecasting/output images/"+county+".png", bbox_inches='tight')
  plt.close()